In [1]:
from flask import Flask, request, jsonify
import xgboost as xgb
import pandas as pd
import pickle
import numpy as np

In [45]:
df=pd.read_csv("data/creditcard_2023.csv")

In [46]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [2]:
app = Flask('fraud')

In [47]:
# Load the trained model and data vectorizer
model_file = 'xgb_model.bin'
with open(model_file, 'rb') as f_in:
    dv, model = pickle.load(f_in)

/Users/muratkahraman/anaconda3/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.3.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:
dv

DictVectorizer(sparse=False)

In [5]:
model

In [48]:
feature_names = [
    'v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9', 'v10',
    'v11', 'v12', 'v13', 'v14', 'v15', 'v16', 'v17', 'v18', 'v19', 'v20',
    'v21', 'v22', 'v23', 'v24', 'v25', 'v26', 'v27', 'v28', 'amount'
]

In [62]:
customer=df.iloc[541].to_dict()

In [63]:
customer

{'id': 541.0,
 'v1': -0.4784271960990545,
 'v2': 0.1421651242876125,
 'v3': -0.0468380289121661,
 'v4': 0.6833502680870056,
 'v5': 0.0678198572410411,
 'v6': -0.4048984594640247,
 'v7': -0.2064959660918449,
 'v8': 0.1843661072753982,
 'v9': -0.7629347043611346,
 'v10': -0.2283918022491541,
 'v11': 0.6609030198474657,
 'v12': -0.3875197498276417,
 'v13': -0.533249442461466,
 'v14': -0.502265671964952,
 'v15': 0.4051430868662198,
 'v16': -0.0606910405034945,
 'v17': -0.2072367443980979,
 'v18': 0.3056026116425647,
 'v19': 0.1348757242981795,
 'v20': -0.0339211457440372,
 'v21': 0.0989766935994606,
 'v22': -0.0751912083426939,
 'v23': -0.4814893372271562,
 'v24': 0.6788996384076292,
 'v25': -0.0115196803518529,
 'v26': 0.4090207868545027,
 'v27': 0.0758593945999199,
 'v28': -0.4471391388557191,
 'amount': 1534.53,
 'class': 1.0}

In [64]:
def predict_model(dv, model, df):
    columns = list(df.columns[1:-1].values)
    dicts = df[columns].to_dict(orient='records')
    X = dv.transform(dicts)
    dtest = xgb.DMatrix(X, feature_names=columns)
    y_pred = model.predict(dtest)
    return y_pred

In [65]:
data = pd.DataFrame([customer])
aligned_data = {feature_name: customer[feature_name] for feature_name in feature_names}
y_pred = predict_model(dv, model,data)

In [67]:
def predict_single(dv, model,customer):
    X = dv.transform([customer])
    y_pred = predict_model(dv, model,data)
    return y_pred[0]

In [68]:
predict_single(dv, model,customer)

0.8655938

In [69]:
import requests
url = 'http://localhost:9696/predict'
response = requests.post(url, json=customer)
result = response.json()
result

{'error': "'str' object has no attribute 'items'"}